In [85]:
import pandas as pd
import numpy as np
import random
import gc
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.optimizers import SGD, RMSprop
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, cross_val_score

# Note types
- 0: nothing
- 1: step
- 2: hold start
- 3: hold/roll end
- 4: roll start
- M: mine

# Classes
- 0: nothing
- 1: one note
- 2: two notes
- 3: three or four notes
- 4: hold start
- 5: roll start
- 6: mine

In [40]:
samples_back_included = 8
num_classes = 7
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, notes, index):
    if index < 0:
        return [0] * num_features
    return beat_features[index]

def get_class_for_index(notes, index):
    if index < 0:
        return [1, 0, 0, 0, 0, 0, 0]
    row = notes[index][0]
    (steps, holds, rolls, mines) = [row.count(char) for char in ['1', '2', '4', 'M']]
    steps += (holds + rolls)
    return [int(i) for i in [steps == 0 and mines == 0, steps == 1, steps == 2, steps > 2, holds > 0, rolls > 0, mines > 0]]
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(key, is_full):
    X = []
    y = []
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            row_y = get_class_for_index(notes, i)
            if is_full or (not (row_y == 0 and random.randint(0, 5) != 0)):
                features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, notes, i - j)]
                features.append(i % 48)
                features.append(get_beat_importance(i))
                features.append(i / 48)
                features.append(num_notes - i / 48)
                X.append(features)
                y.append(row_y)
    return np.array(X), np.array(y)

# Total 243 songs
def build_training_data(songs_start, songs_end, is_full = False):
    X = []
    y = []
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use[songs_start:songs_end]:
        song_X, song_y = get_features_for_song(song_data[0], is_full)
        X.extend(song_X)
        y.extend(song_y)
    return X, y

In [41]:
X, y = build_training_data(0, 243, True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X = None
y = None
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)

In [42]:
class_weight = {
    0: 1,
    1: 2,
    2: 4,
    3: 8,
    4: 4,
    5: 4,
    6: 6
}

In [87]:
def train_and_test(beat_feature_model):
    print ('Train Error')
    print (beat_feature_model.evaluate(X_train, y_train, batch_size=64))
    print ('Test Error')
    print (beat_feature_model.evaluate(X_test, y_test, batch_size=64))
    print ('\n')

In [127]:
model = Sequential()

model.add(Dense(512, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=50, batch_size=64, verbose=1, validation_data=(X_test, y_test))
train_and_test(model)
model.save('models/song_class_model_alt.h5')
model = None
gc.collect()

Train on 481012 samples, validate on 120253 samples
Epoch 1/50
481012/481012 [==============================] - 146s - loss: 0.8557 - acc: 0.8430 - val_loss: 0.5917 - val_acc: 0.8689
Epoch 2/50
481012/481012 [==============================] - 134s - loss: 0.5401 - acc: 0.8700 - val_loss: 0.4977 - val_acc: 0.8712
Epoch 3/50
481012/481012 [==============================] - 142s - loss: 0.4769 - acc: 0.8725 - val_loss: 0.4514 - val_acc: 0.8753
Epoch 4/50
481012/481012 [==============================] - 124s - loss: 0.4498 - acc: 0.8736 - val_loss: 0.4335 - val_acc: 0.8761
Epoch 5/50
481012/481012 [==============================] - 133s - loss: 0.4347 - acc: 0.8746 - val_loss: 0.4222 - val_acc: 0.8751
Epoch 6/50
481012/481012 [==============================] - 129s - loss: 0.4240 - acc: 0.8750 - val_loss: 0.4138 - val_acc: 0.8785
Epoch 7/50
481012/481012 [==============================] - 136s - loss: 0.4171 - acc: 0.8755 - val_loss: 0.4072 - val_acc: 0.8786
Epoch 8/50
481012/481012 [=====

121183

In [ ]:
model = Sequential()

model.add(Dense(512, input_shape=(324,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

model.fit(X_train, y_train, nb_epoch=50, batch_size=64, verbose=1, class_weight=class_weight, validation_data=(X_test, y_test))
train_and_test(model)
model.save('models/song_class_model_weighted.h5')
model = None
gc.collect()

Train on 481012 samples, validate on 120253 samples
Epoch 1/50
447168/481012 [==========================>...] - ETA: 8s - loss: 1.2282 - acc: 0.8046